# Model selection - drop questionnaire items

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5019184/

## Environment initialization

In [16]:
from __future__ import division, print_function

%autosave 0
%matplotlib notebook
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import sklearn as sk
from sklearn import preprocessing
import pandas as pd

import sys
sys.path.append("../")

import mod_data
import mod_evaluation
import mod_compute
import mod_viewer
import mod_latent

Autosave disabled
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Notebook variables

In [17]:
data_path = '../data/source/all_data_wide.csv'

metric_eval = 'val_categorical_accuracy'

## Execution params

In [18]:
results_path = 'data/results'

q_exc = []

metric_id = 'categorical_accuracy_class'

model_ref_id = 'linear'
model_ref_kwargs = {}

scaling = None

n_splits = 25
drop_max = 120

train = 0
test_size = 200

## Running params

In [19]:
model_ref = mod_compute.models[model_ref_id]
cache_pre = 'model_'+model_ref_id

## Model search

In [20]:
for train_val_random in range(50):
    
    cache_post = str(n_splits) + '_r'+str(train_val_random)

    cache_sig = mod_evaluation.cache_sig_gen(
        metric_id, 
        cache_pre=cache_pre,
        cache_post=cache_post
    )

    print('evaluating', cache_sig)

    df_data = pd.read_csv(data_path)

    _, logits = mod_latent.modelPP(df_data[mod_latent.s_latent_ref])

    df_data[mod_data.logits_ref] = pd.DataFrame(logits)
    
    df_data = df_data[df_data['train']==train]
    
    df_train, df_val = sk.model_selection.train_test_split(
        df_data,
        test_size=test_size,
        random_state=train_val_random,
        shuffle=True,
        stratify=np.argmax(np.array(df_data[mod_data.logits_ref]), axis=1)
    )

    df_train_val = (df_train, df_val)

    df_x_train, df_y_train, df_x_val, df_y_val = mod_data.load_data_df(df_train_val=df_train_val)

    stats_best, info_best = mod_compute.stats_seq_drop(
        metric_id,
        model_ref,
        df_x_train,
        df_y_train,
        q_exc_init=q_exc, 
        drop_max=drop_max,
        n_splits=n_splits,
        metric_eval=metric_eval,
        **model_ref_kwargs
    )

    ## Model validation
    stats_val = {}

    for model_id in info_best:

        q_sel = mod_compute.get_q_sel(info_best[model_id]['q_exc'])

        x_train, y_train, x_val, y_val = mod_compute.df_to_model_input(
            df_x_train[q_sel], df_y_train, df_x_val[q_sel], df_y_val
        )

        _, my_stats =  mod_compute.model_train(
            x_train, y_train, x_val, y_val, 
            model_ref,
            **model_ref_kwargs
        )

        stats_val[model_id] = [my_stats]

    ## Save results
    if not os.path.isdir(results_path):
        os.mkdir(results_path)

    results = {
        'info': info_best,
        'stats': stats_best,
        'stats_val': stats_val,
    }

    mod_evaluation.to_cache(cache_sig, results_path, results)

evaluating model_linear_categorical_accuracy_class_25_r0


drop_q	 drop_sum	val_acc_m (val_acc_min - val_acc_max)	 name



ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/share/tools3/lib/python3.5/site-packages/pandas/core/indexes/base.py", line 2657, in get_loc
    return self._engine.get_loc(key)
  File "pandas/_libs/index.pyx", line 108, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 132, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 1601, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 1608, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'PANAS_negative_affect'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/share/tools3/lib/python3.5/site-packages/pandas/core/internals/managers.py", line 1053, in set
    loc = self.items.get_loc(item)
  File "/share/tools3/lib/python3.5/site-packages/pandas/core/indexes/base.py", line 2659, in get_loc
    return self._engine.get_loc(self._maybe_cast

TypeError: Can't convert 'list' object to str implicitly